In [2]:
import os

import demeter

import numpy as np
import pandas as pd


In [3]:
data_dir = "/Users/d3y010/projects/demeter/chen/demeter_inputs_chen_et_al_2020a"

observed_dir = os.path.join(data_dir, 'inputs', 'observed')
projected_dir = os.path.join(data_dir, 'inputs', 'projected')
constraints_dir = os.path.join(data_dir, 'inputs', 'constraints')

base_layer_file = os.path.join(observed_dir, "baselayerdata_region_basin_0.05deg.csv")
out_base_layer_file = os.path.join(observed_dir, "conus_baselayerdata_region_basin_0.05deg.csv")

proj_file = os.path.join(projected_dir, "ssp1_rcp45_ipsl_fordownscaling.csv")
out_proj_file = os.path.join(projected_dir, "conus_ssp1_rcp45_ipsl_fordownscaling.csv")

nut_avail_file = os.path.join(constraints_dir, 'orig_000_nutrientavail_hswd_NA_0.05.csv')
out_nut_avail_file = os.path.join(constraints_dir, '000_nutrientavail_hswd_NA_0.05.csv')

soil_file = os.path.join(constraints_dir, 'orig_001_soilquality_hswd_0.05.csv')
out_soil_file = os.path.join(constraints_dir, '001_soilquality_hswd_0.05.csv')

config_file = os.path.join(data_dir, 'conus_config.ini-ssp1_rcp45_ipsl_fordownscaling')


In [4]:
bdf = pd.read_csv(base_layer_file)


In [9]:
bdf.columns


Index(['Latcoord', 'Loncoord', 'water', 'PFT1', 'PFT2', 'PFT3', 'PFT4', 'PFT5',
       'PFT6', 'PFT7', 'PFT8', 'PFT9', 'PFT10', 'PFT11', 'PFT12', 'PFT13',
       'PFT14', 'PFT15', 'PFT16', 'PFT17', 'PFT18', 'PFT19', 'PFT20', 'PFT21',
       'PFT22', 'PFT23', 'PFT24', 'PFT25', 'PFT26', 'PFT27', 'PFT28', 'PFT29',
       'PFT30', 'PFT31', 'PFT32', 'PFT33', 'PFT34', 'PFT35', 'PFT36', 'PFT37',
       'PFT38', 'PFT39', 'PFT40', 'PFT41', 'PFT42', 'PFT43', 'PFT44', 'PFT45',
       'PFT46', 'PFT47', 'PFT48', 'PFT49', 'PFT50', 'PFT51', 'PFT52', 'PFT53',
       'PFT54', 'PFT55', 'PFT56', 'PFT57', 'PFT58', 'PFT59', 'PFT60', 'PFT61',
       'PFT62', 'PFT63', 'PFT64', 'PFT65', 'PFT66', 'PFT67', 'PFT68', 'PFT69',
       'PFT70', 'PFT71', 'PFT72', 'PFT73', 'PFT74', 'PFT75', 'PFT76', 'PFT77',
       'PFT78', 'PFT79', 'region_id', 'basin_id', 'fid'],
      dtype='object')

In [8]:
sorted(bdf.region_id.unique())

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31]

## Reformat the input data to run sequentially by basin id

### Only keep the basins that are in the CONUS

In [28]:
conus_basin_list = [23, 27, 96, 212, 213, 214, 215, 216, 
                    217, 218, 219, 220, 221, 222, 223, 
                    224, 225, 226, 227, 228]


In [47]:
# build dict of sequential assignment to each basin id
conus_basin_reorder = {i: index for index, i in enumerate(sorted(conus_basin_list), 1)}


### Base layer

In [60]:
# build dict of sequential assignment to each fid
conus_fid_reorder = {i: index for index, i in enumerate(sorted(usf['fid'].values), 1)}


In [64]:
usf = bdf.loc[(bdf['region_id'] == 1) & (bdf['basin_id'].isin(conus_basin_list))].copy()

# rename the existing basin_id to be sequential
usf['orig_basin_id'] = usf['basin_id']

usf['basin_id'] = usf['orig_basin_id'].map(conus_basin_reorder)

# rename the existing basin_id to be sequential
usf['orig_fid'] = usf['fid']

usf['fid'] = usf['orig_fid'].map(conus_fid_reorder)

usf.to_csv(out_base_layer_file, index=False)


### Projected data

In [75]:
pdf = pd.read_csv(proj_file)

pdf = pdf.loc[(pdf['region'] == "USA") & (pdf["metric_id"].isin(conus_basin_list))]

# rename the existing basin_id to be sequential
pdf['orig_metric_id'] = pdf['metric_id']

pdf['metric_id'] = pdf['orig_metric_id'].map(conus_basin_reorder)

pdf.to_csv(out_proj_file, index=False)

pdf.head()


,region,landclass,metric_id,1975,1990,2005,2010,2015,2020,2025,...,2065,2070,2075,2080,2085,2090,2095,2100,Units,orig_metric_id
0,USA,biomass-grass_IRR,1,0.0,0.0,0.0,0.0,0.0,0.004148,0.006625,...,0.033583,0.035399,0.035369,0.036474,0.036631,0.037527,0.038494,0.037726,thous km2,23
1,USA,biomass-grass_IRR,2,0.0,0.0,0.0,0.0,0.0,0.000077,0.000121,...,0.000447,0.000433,0.000389,0.000363,0.000331,0.000287,0.000251,0.000218,thous km2,27
2,USA,biomass-grass_IRR,3,0.0,0.0,0.0,0.0,0.0,0.000408,0.000656,...,0.002321,0.002299,0.002751,0.003155,0.003368,0.003354,0.003580,0.004339,thous km2,96
4,USA,biomass-grass_IRR,4,0.0,0.0,0.0,0.0,0.0,0.220160,0.351013,...,1.784421,1.934343,2.072440,2.226120,2.289190,2.402560,2.548590,2.731680,thous km2,212
5,USA,biomass-grass_IRR,5,0.0,0.0,0.0,0.0,0.0,0.005857,0.009720,...,0.051175,0.052860,0.055401,0.057011,0.057567,0.057745,0.057548,0.057191,thous km2,213


### Constraints

In [73]:
ndf = pd.read_csv(nut_avail_file, header=None, names=['fid', 'weight'])

# rename the existing basin_id to be sequential
ndf['fid'] = ndf['fid'].map(conus_fid_reorder)

ndf.to_csv(out_nut_avail_file, index=False, header=False)


In [74]:
sdf = pd.read_csv(soil_file, header=None, names=['fid', 'weight'])

# rename the existing basin_id to be sequential
sdf['fid'] = sdf['fid'].map(conus_fid_reorder)

sdf.to_csv(out_soil_file, index=False, header=False)


## Run demeter

In [3]:
demeter.run_model(config_file=config_file)


2022-01-19 14:52:39,924 - demeter_runtime - INFO - Using `observed_lu_file`:  /Users/d3y010/projects/demeter/chen/demeter_inputs_chen_et_al_2020a/inputs/observed/baselayerdata_region_basin_0.05deg.csv
2022-01-19 14:52:39,926 - demeter_runtime - INFO - Using `run_dir`:  /Users/d3y010/projects/demeter/chen/demeter_inputs_chen_et_al_2020a
2022-01-19 14:52:39,927 - demeter_runtime - INFO - START
2022-01-19 14:52:39,943 - demeter_runtime - INFO - Reading allocation input files...
2022-01-19 14:52:39,988 - demeter_runtime - INFO - PERFORMANCE:  Allocation files processed in 0.04355502128601074 seconds
2022-01-19 14:52:39,991 - demeter_runtime - INFO - Preparing projected land use data...
2022-01-19 14:52:39,992 - demeter_runtime - INFO - Using projected GCAM data from:  /Users/d3y010/projects/demeter/chen/demeter_inputs_chen_et_al_2020a/inputs/projected/ssp1_rcp45_ipsl_fordownscaling.csv
2022-01-19 14:52:40,100 - demeter_runtime - INFO - Number of regions from projected file:  32
2022-01-19 

AttributeError: 'NoneType' object has no attribute 'error'